# Assignment 9

In [1]:
import numpy as np
import pandas as pd

### Part A

Consider a two period binomial model. Suppose after each period, the stock will either rise in value by ```r``` percent, or fall in value by ```f``` percent. Currently, the stock price is ```P```, and the risk free rate is ```rf```.

Write the function ```european_put```, which calculates the value of a one year European Put option with a strike price of ```K```. 

In [2]:
def european_put(P, K, r, f, rf):
    rf_rate = 1 + rf
    
    u_rate = 1 + r
    
    d_rate = 1 + f
    
    Suu = P * (u_rate ** 2)
    
    Sud = P * (u_rate * d_rate)
    
    Sdd = P * (d_rate ** 2)
    
    risk_neutral_rise = ((np.sqrt(rf_rate) - 1) - f) / (r - f)
    
    risk_neutral_fall = 1 - risk_neutral_rise
    
    Price_1 = ((risk_neutral_rise * 0) + risk_neutral_fall * (K - Sdd)) / np.sqrt(rf_rate)
    
    Price_0 = ((risk_neutral_rise * 0) + risk_neutral_fall * Price_1) / np.sqrt(rf_rate)
    
    return Price_0
            
# european_put(40,40, 0.25, -0.2, 0.1) should return 2.616...

Write the function ```american_put```, which calculates the value of a one year American Put option with a strike price of ```K```. 

In [3]:
def american_put(P, K, r, f, rf):
    
    rf_rate = 1 + rf
    
    d_rate = 1 + f
    
    Sd = P * d_rate
    
    Num = (np.sqrt(rf_rate) - 1) - f
    
    Denom = r - f
    
    R_Neutral_Rise = Num / Denom
    
    R_Neutral_Fall = 1 - R_Neutral_Rise
    
    Pd = K - Sd
    
    P0 = ((R_Neutral_Rise * 0) + R_Neutral_Fall * Pd) / np.sqrt(rf_rate)
    
    return P0

# american_put(40,40, 0.25, -0.2, 0.1) should return 3.410...

### Part B

In addition to simple calls and puts, a wide variety of exotic options are traded in over-the-counter markets. In most cases, there are no analytic formulas available to value these options, and so numerical techniques must be used. One example of these techniques are Monte Carlo methods. These are particularly useful in certain contexts, such as when option payoffs depend not only on the stock price at expiration but also upon the history of the stock price sampled at various points during the life of the option. This question will explore some basic examples, and it will require relatively large number of paths. Assume that the stock price will be needed at the end of each of the next 12 months, in addition to S0 today, and start by simulating `N` such stock price paths. In particular:

•	You have been provided the function ```gen_random_vars```, which generates random variables that follow a standard normal distribution (Note that you should never actually call ```gen_random_vars``` in any of your functions, this is simply meant to help give you inputs to help test your code). This outputs a table with `N` rows and 12 columns. Denote the value of the random variable on path i for month j by zi,j where i = 1, . . . ,`N` and j = 1, . . . ,12.

Write the function ```create_paths```, which takes as an argument a DataFrame ```random_vars```, the output of the function ```gen_random_vars```, and ```S0```, the initial stock price.

This function will generate a DataFrame with `N` stock price paths $S_{i,j}$ where $i = 1, . . . ,N$ and $j = 0, . . . ,12$. In each case, $S_{i,0} =$ ```S_0```. For the remaining entries in this table, define $∆t = 1/12$ and set

$$ S_{i,j} = S_{i, j-1} \exp [(r - \sigma^2 / 2) \Delta t + \sigma \sqrt{\Delta t} z_{i,j}]$$

 
for i = 1, . . . ,`N` and j = 1, . . . ,12. (If you are wondering why we are using the risk-free rate r here instead of the expected rate of return on the underlying stock, it is because we are using risk-neutral valuation.) 

Note that throughout the rest of this question, $S_T$ on each simulated path i is defined to be equal to $S_{i,12}$.



In [4]:
def gen_random_vars(SEED, N):
    rng = np.random.default_rng(SEED)

    zArr = np.array([rng.normal(size=N)])
    for i in range(11):
        z = rng.normal(size=N)
        zArr = np.r_[zArr,[z]]
    

    randomVariables = pd.DataFrame(zArr.transpose())
    return randomVariables

r_vars = gen_random_vars(727, 10000)
r_vars

0         1         2         3         4         5         6   \
0     1.744763 -0.514250 -0.710679  1.495061  0.413633 -2.361391  0.122873   
1    -0.962309 -0.576888  0.396051  1.595821  0.008168 -0.348428 -0.340074   
2     0.496945  0.579497 -0.315875  0.330535 -0.045317  0.722330  0.760781   
3    -0.282490 -0.940921  1.175841  0.405341 -0.923402  0.977314 -0.128823   
4    -0.400287 -1.035328 -1.143469 -0.976739  0.649610 -0.414282 -0.257923   
...        ...       ...       ...       ...       ...       ...       ...   
9995  0.812176  0.492083  0.234801 -0.822796 -1.171159 -1.472986  0.598065   
9996 -0.714678 -0.710493 -0.644405  0.711664 -0.901012  0.526393 -2.218868   
9997 -1.684857  0.816217  0.048860  0.366472  0.296956 -0.250303  1.962326   
9998 -0.418933  0.839082 -2.012594  1.065439 -0.527060 -0.770415 -0.886027   
9999 -0.471454  0.268310  0.767433 -1.018053 -1.031914 -0.759433  0.806048   

            7         8         9         10        11  
0    -0.095099 -0.884679 -0.148495 -0.508705  1.562580  
1     0.230534 -0.556868 -1.567693 -0.009760  0.885091  
2    -0.070977  0.250281 -0.329167  0.407775  0.864023  
3    -0.688989 -1.411402  0.010026 -1.573026  0.730829  
4     0.762621 -1.664448 -1.159700  0.268761  1.822126  
...        ...       ...       ...       ...       ...  
9995 -0.239307 -0.822385  0.653382  0.729384  0.928624  
9996  2.422271  0.392027  1.610234  0.387898 -0.058958  
9997  1.416475  0.541789  0.660781  1.132385  0.190961  
9998  0.911825 -0.648514 -2.227339 -0.125608 -1.135578  
9999  0.445378  0.758105  0.378315 -0.898852  0.473172  

[10000 rows x 12 columns]

In [5]:
def create_paths(random_vars, S0, rf, sigma):
    # Do not rename the columns or rows of the outputted dataframe. 
    # The outputted DataFrame should have N rows and 13 columns.
    monthly_incr = 1/12
    
    N = len(random_vars)
    
    paths = pd.DataFrame(index=range(N), columns=range(13))
    
    for i in range(13):
        if (i == 0):
            paths[i] = S0
        else:
            current_random_var = random_vars[i-1]
            
            component_a = (rf - pow(sigma, 2)/2) * monthly_incr
            
            component_b = sigma * np.sqrt(monthly_incr) * current_random_var
            
            paths[i] = paths[i-1] * np.exp(component_a + component_b)
    
    return paths

# If we let sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35),
# The last column of the first row of sample_paths should equal 71.56...

•	Write the function ```payoff_table```, which returns another DataFrame with `N` rows and 6 columns. The columns will be used to store discounted payoffs at expiration for various different options, as indicated below:

–	European call option, with payoff $max(S_T − K,0)$.

–	European put option, with payoff $max(K - S_T ,0)$.

–	Average price put option, with payoff $max(K − S^-,0)$, where
$S¯ = S_0 + S_1 + · · · + S_{12}$ .

–	Floating lookback call option, with payoff $S_T - S_{min}$, where $S_{min}$ on each path is the minimum of the stock prices $S_0, S_1, . . . , S_{12}$ on that path.

–	Up-and-out call option with barrier ```B```, with payoff $max(S_T - K,0)$ as long as none of the observed monthly stock prices on the given path are greater than or equal to ```B```. If any of the monthly stock prices are greater than or equal to B, then the payoff is zero.

–	Up-and-in call option with barrier ```B``` , with payoff $max(S_T - K,0)$ as long as at least one of the observed monthly stock prices on the given path is greater than or equal to ```B```. If this condition is not satisfied, the option payoff is zero.

Note that “discounted payoffs” implies that the entries in this table should have the payoff functions multiplied by $e^{−rT}$ , where T = 1.

This function will also take in a dataframe ```paths```, which is the output from the previous function ```create_paths```.

In [6]:
def payoff_table(paths, K, T, rf, B):
    Discount_Payoffs = np.exp(T * -rf)
    
    European_Call_Option = paths.iloc[:, -1].apply(lambda x: max(x - K, 0) * Discount_Payoffs)
    
    European_Put_Option = paths.iloc[:, -1].apply(lambda x: max(K - x, 0) * Discount_Payoffs)
    
    Average_Price_Put_Option = (K - paths.mean(axis=1)).apply(lambda x: max(x, 0) * Discount_Payoffs)
    
    S_min = paths.min(axis=1)
    
    Floating_Lookback_Call_Option = (paths.iloc[:, -1] - S_min).apply(lambda x: max(x, 0) * Discount_Payoffs)
    
    Up_and_Out = np.where(paths.max(axis=1) < B, paths.iloc[:, -1].apply(lambda x: max(x - K, 0) * Discount_Payoffs), 0)
    
    Up_and_In = np.where(paths.max(axis=1) >= B, paths.iloc[:, -1].apply(lambda x: max(x - K, 0) * Discount_Payoffs), 0)
    
    payoffs = pd.DataFrame({
        "European Call": European_Call_Option,
        "European Put": European_Put_Option,
        "Average Price Put": Average_Price_Put_Option,
        "Floating Lookback": Floating_Lookback_Call_Option,
        "Up-and-out": Up_and_Out,
        "Up-and-in": Up_and_In
    })
    
    return payoffs
            
# If we let sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
# The payoff of the first European Call should be 1.500...

Write the function ```statistics```, which takes as input a DataFrame ```payoffs```, formatted as one produced by ```payoff_table```, and displays the following statistics for each of the options above (below N is the number of simulated paths):

•	Mean discounted payoff µ. This is the estimated option premium.

•	Standard error of option premium ϕ. This is the standard deviation of the option payoff divided by √N .

•	Lower bound for 95% confidence interval for option premium, which is µ - 1.96ϕ.

•	Upper bound for 95% confidence interval for option premium, which is µ + 1.96ϕ.

The size of the returned DataFrame should have 6 rows and 4 columns. 

In [7]:
def statistics(payoffs):
    
    Std_Error = payoffs.sem()
    
    Confidence_Interval = 1.96 * Std_Error
    
    Mean_Payoff = payoffs.mean()
    
    
    # Note that the index rows have been renamed here. Ensure that your rows match the order 
    # that the indices have been renamed as.
    output = pd.DataFrame(data = {
        "Mean" : Mean_Payoff.values,
        "Standard Error" : Std_Error.values,
        "Lower Bound" : (Mean_Payoff - Confidence_Interval).values,
        "Upper Bound" : (Mean_Payoff + Confidence_Interval).values
    }).rename(index={0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}, errors="raise")
    
    return output

# If we call statistics(sample_payoffs), the mean of the European Puts should be 7.370...

One disadvantage of Monte Carlo methods is that they can be slow to converge. On the other hand, there are situations where they are the only methods that can be used. As a result, there has been a lot of research into variance reduction methods, in an effort to improve the rate of convergence. 

One of the simplest such approaches is called antithetic variates. This relies on the fact that if Z is a standard normal random variable, then so is -Z. To see how this can be used, generate another `N` stock price paths, but this time using $-z_{i,j}$ instead of $z_{i,j}$ throughout.

In order to accomplish this, write the function ```create_negative_paths```. This function has the same arguments as ```create_paths```, but creates the stock prices in the manner described above.

In [8]:
def create_negative_paths(random_vars, S0, rf, sigma):
    # Write your code here
    monthly_incr = 1/12
    
    N = len(random_vars)
    
    paths = pd.DataFrame(index=range(N), columns=range(13))
    
    for i in range(13):
        if (i == 0):
            paths[i] = S0
        else:
            current_random_var = (random_vars[i-1]) * -1
            
            component_a = (rf - pow(sigma, 2)/2) * monthly_incr
            
            component_b = sigma * np.sqrt(monthly_incr) * current_random_var
            
            paths[i] = paths[i-1] * np.exp(component_a + component_b)
    
    return paths

Write the function ```antithetic_statistics```.

In this function, you will use ```payoff_table``` to create two tables, one using paths generated from ```create_paths```, and one generated from ```create_negative_paths```.

Next, create a third table which contains the average payoff between the two tables.

Finally, call ```statistics``` on this third table. Return the table produced by ```statistics```.

If you compare this table with the output of `statistics` (as above, without creating two tables), you should notice that Mean, Lower Bound, and Upper Bound columns should have relatively similar values, while the Standard Error column has decreased significantly.

In [9]:
def antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    
    Postive_Path = create_paths(random_vars, S0, rf, sigma)
    
    Negative_Path = create_negative_paths(random_vars, S0, rf, sigma)
    
    Positive_Payoffs = payoff_table(Postive_Path, K, T, rf, B)
    
    Negative_Payoffs = payoff_table(Negative_Path, K, T, rf, B)
    
    Total_Payoffs = Positive_Payoffs + Negative_Payoffs
    
    Average_Payoffs = Total_Payoffs / 2
    
    Stats = statistics(Average_Payoffs)
    
    return Stats